In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.utils import resample

In [2]:
data_dir = './data/csv'

data_entry_file = os.path.join(data_dir, 'Data_Entry_2017_v2020.csv')
sample_labels_file = os.path.join(data_dir, 'sample_labels.csv')

data_entry_df = pd.read_csv(data_entry_file)
sample_labels_df = pd.read_csv(sample_labels_file)

In [3]:
data_entry_df_copy = data_entry_df.copy()
sample_labels_df_copy = sample_labels_df.copy()

In [4]:
finding_labels_df = data_entry_df_copy['Finding Labels'].str.get_dummies(sep='|')

data_entry_df_copy = pd.concat([data_entry_df_copy, finding_labels_df], 1)

In [5]:
old_new_columns_map = {
    'Image Index': 'image_index',
    'Finding Labels': 'finding_labels',
    'Follow-up #': 'follow_up_number',
    'Patient ID': 'patient_id',
    'Patient Age':'patient_age',
    'Patient Gender': 'patient_gender',
    'View Position': 'view_position',
    'OriginalImage[Width': 'original_image_width',
    'Height]': 'original_image_height',
    'OriginalImagePixelSpacing[x': 'original_image_pixel_spacing_x',
    'y]': 'original_image_pixel_spacing_y',
    'Atelectasis': 'atelectasis',
    'Cardiomegaly': 'cardiomegaly',
    'Consolidation': 'consolidation',
    'Edema': 'edema',
    'Effusion': 'effusion',
    'Emphysema': 'emphysema',
    'Fibrosis': 'fibrosis',
    'Hernia': 'hernia',
    'Infiltration': 'infiltration',
    'Mass': 'mass',
    'No Finding': 'no_finding',
    'Nodule': 'nodule',
    'Pleural_Thickening': 'pleural_thickening',
    'Pneumonia': 'pneumonia',
    'Pneumothorax': 'pneumothorax'
}

data_entry_df_copy.rename(columns=old_new_columns_map, inplace=True)
sample_labels_df_copy.rename(columns=old_new_columns_map, inplace=True)

In [6]:
old_new_columns_map = {
    'Atelectasis': 'atelectasis',
    'Cardiomegaly': 'cardiomegaly',
    'Consolidation': 'consolidation',
    'Edema': 'edema',
    'Effusion': 'effusion',
    'Emphysema': 'emphysema',
    'Fibrosis': 'fibrosis',
    'Hernia': 'hernia',
    'Infiltration': 'infiltration',
    'Mass': 'mass',
    'No Finding': 'no_finding',
    'Nodule': 'nodule',
    'Pleural_Thickening': 'pleural_thickening',
    'Pneumonia': 'pneumonia',
    'Pneumothorax': 'pneumothorax'
}

finding_labels_df.rename(columns=old_new_columns_map, inplace=True)

In [7]:
sample_size = data_entry_df_copy.shape[0]

pneumonia_cases = data_entry_df_copy.query('pneumonia == 1')
nonpneumonia_cases = data_entry_df_copy.query('pneumonia == 0')

num_pneumonia_cases = pneumonia_cases.shape[0]
num_nonpneumonia_cases = nonpneumonia_cases.shape[0]

pct_pneumonia_cases = num_pneumonia_cases / sample_size
pct_nonpneumonia_cases = num_nonpneumonia_cases / sample_size

In [8]:
print('Total number of records: {}'.format(sample_size))

print('Total number of Pneumonia cases: {}'.format(num_pneumonia_cases))
print('Total number of non Pneumonia cases: {}'.format(num_nonpneumonia_cases))

print('Percentage of Pneumonia: {:0.2f}'.format(pct_pneumonia_cases))
print('Percentage of non Pneumonia: {:0.2f}'.format(pct_nonpneumonia_cases))

Total number of records: 112120
Total number of Pneumonia cases: 1431
Total number of non Pneumonia cases: 110689
Percentage of Pneumonia: 0.01
Percentage of non Pneumonia: 0.99


---
---
---

In [9]:
final_size = sample_labels_df_copy.shape[0]

size = int(final_size / 2)

In [10]:
df_minority = pneumonia_cases.copy()

In [11]:
df_majority = nonpneumonia_cases[nonpneumonia_cases.image_index.isin(sample_labels_df_copy.image_index)]

In [12]:
df_minority_upsampled = resample(df_minority, 
                                 replace=True,       # sample with replacement
                                 n_samples=size,    # to match majority class
                                 random_state=123)   # reproducible results

In [13]:
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=size,     # to match minority class
                                 random_state=123) # reproducible results

In [14]:
df_balanced = pd.concat([df_majority_downsampled, df_minority_upsampled])

df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)

---
---
---

In [15]:
training_size = int(np.rint(final_size * 0.8))
testing_size = final_size - training_size

training_size, testing_size

(4485, 1121)

In [16]:
df_training = df_balanced.sample(training_size)

In [17]:
df_testing = df_balanced[~df_balanced.index.isin(df_training.index)]

In [18]:
df_training = df_training.sample(frac=1).reset_index(drop=True)
df_testing = df_testing.sample(frac=1).reset_index(drop=True)

In [19]:
df_training

,image_index,finding_labels,follow_up_number,patient_id,patient_age,patient_gender,view_position,original_image_width,original_image_height,original_image_pixel_spacing_x,...,emphysema,fibrosis,hernia,infiltration,mass,no_finding,nodule,pleural_thickening,pneumonia,pneumothorax
0,00009736_016.png,Pneumothorax,16,9736,65,F,PA,2992,2991,0.143,...,0,0,0,0,0,0,0,0,0,1
1,00021489_013.png,Infiltration|Pneumonia,10,21489,31,M,AP,3056,2544,0.139,...,0,0,0,1,0,0,0,0,1,0
2,00007018_007.png,No Finding,7,7018,21,M,AP,2500,2048,0.171,...,0,0,0,0,0,1,0,0,0,0
3,00021584_001.png,Pneumonia,8,21584,46,M,PA,2992,2991,0.143,...,0,0,0,0,0,0,0,0,1,0
4,00026818_017.png,Edema|Pneumonia,18,26818,31,M,AP,3056,2544,0.139,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4480,00026908_002.png,Pneumonia,2,26908,63,M,AP,3056,2544,0.139,...,0,0,0,0,0,0,0,0,1,0
4481,00012141_022.png,Edema|Effusion|Pleural_Thickening|Pneumonia,22,12141,31,M,AP,2500,2048,0.168,...,0,0,0,0,0,0,0,1,1,0
4482,00002048_013.png,Consolidation,10,2048,43,M,PA,2986,2991,0.143,...,0,0,0,0,0,0,0,0,0,0
4483,00019894_038.png,Pneumonia,38,19894,61,M,AP,3056,2544,0.139,...,0,0,0,0,0,0,0,0,1,0


In [20]:
df_testing

,image_index,finding_labels,follow_up_number,patient_id,patient_age,patient_gender,view_position,original_image_width,original_image_height,original_image_pixel_spacing_x,...,emphysema,fibrosis,hernia,infiltration,mass,no_finding,nodule,pleural_thickening,pneumonia,pneumothorax
0,00021710_002.png,Pneumonia,1,21710,59,M,PA,2992,2991,0.143000,...,0,0,0,0,0,0,0,0,1,0
1,00016406_004.png,Edema|Effusion|Infiltration|Pneumonia,4,16406,42,F,AP,2500,2048,0.168000,...,0,0,0,1,0,0,0,0,1,0
2,00013844_004.png,Atelectasis,2,13844,36,M,AP,2500,2048,0.168000,...,0,0,0,0,0,0,0,0,0,0
3,00001347_002.png,No Finding,1,1347,46,F,PA,2602,2991,0.143000,...,0,0,0,0,0,1,0,0,0,0
4,00004512_002.png,Atelectasis|Consolidation|Infiltration|Pneumonia,2,4512,11,M,PA,2048,2500,0.171000,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,00014346_024.png,Infiltration,9,14346,19,F,AP,3036,2544,0.139000,...,0,0,0,1,0,0,0,0,0,0
1117,00030039_019.png,No Finding,19,30039,52,M,PA,2021,2021,0.194311,...,0,0,0,0,0,1,0,0,0,0
1118,00011704_000.png,Effusion|Pneumonia,0,11704,24,F,PA,2992,2991,0.143000,...,0,0,0,0,0,0,0,0,1,0
1119,00004526_013.png,Atelectasis|Infiltration|Pneumonia,3,4526,35,F,PA,2992,2991,0.143000,...,0,0,0,1,0,0,0,0,1,0


In [21]:
training_filepath = os.path.join(data_dir, "train_data.csv")
df_training.to_csv (training_filepath, index = True, header = True)

testing_filepath = os.path.join(data_dir, "test_data.csv")
df_testing.to_csv (testing_filepath, index = True, header = True)